In the two notebooks statistical_Modeling.ipynb and data_exploration_analysis.ipynb we explored the data, analyze it, studying correlation and and building statistical models that could forecast the weekly_sales of each store.

In this notebook, I will build ML based models for forecasting the weekly sales and see if they can outperformed the statistical models.

In [ ]:
!git clone https://github.com/HUMANITICS/ml-test.git
%cd ml-test

/content/drive/MyDrive/store_retail/ml-test


In [ ]:
import random
from typing import Union
from tqdm import tqdm_notebook
from itertools import product
import warnings


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor , VotingRegressor



from tensorflow.keras.losses import MeanSquaredError , MeanAbsoluteError , Huber
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [ ]:
df = pd.read_csv("stores_sales.csv")
print(f"The shape of this data is {df.shape}")
df.head(2)

The shape of this data is (6435, 8)


,store,date,weekly_sales,holiday_flag,temperature,fuel_Price,cpi,unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106


In [ ]:
df.index = pd.to_datetime(df['date'] , format = "%d-%m-%Y")
df.drop(["date"] , axis=1 , inplace=True)
df.head(3)

,store,weekly_sales,holiday_flag,temperature,fuel_Price,cpi,unemployment
date,,,,,,,
2010-02-05,1,1643690.90,0,42.31,2.572,211.096358,8.106
2010-02-12,1,1641957.44,1,38.51,2.548,211.242170,8.106
2010-02-19,1,1611968.17,0,39.93,2.514,211.289143,8.106


**1.  Machine Learning based approaches**

To initiate the process, we'll commence with data partitioning. Specifically, we'll allocate 80% of the available data for training purposes, while the remaining 20% will serve as the testing subset. This division will entail segregating 80% of the data from each of the 45 available stores for training, while the complementary portion will be reserved for testing.

In [ ]:
warnings.filterwarnings("ignore")


rows_train = 115
rows_test = 28

train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

# Iterate over unique store values
for store_id in df['store'].unique():
    store_data = df[df['store'] == store_id]

    # Some feature engineering
    store_data["weekly_sales(t-1)"] , store_data["weekly_sales(t-2)"] = store_data["weekly_sales"].shift(1) , store_data["weekly_sales"].shift(2)
    store_data["weekly_sales(t-3)"] , store_data["weekly_sales(t-4)"] = store_data["weekly_sales"].shift(3) , store_data["weekly_sales"].shift(4)

    # Split data for train and test
    train_data = store_data.head(rows_train)
    test_data = store_data.tail(rows_test)


    # Append to train_df and test_df
    train_df = train_df.append(train_data)
    test_df = test_df.append(test_data)

del train_data , test_data

In [ ]:
store_id = 1
len(train_df[train_df["store"] == store_id]) == rows_train , len(test_df[test_df["store"] == store_id]) == rows_test

(True, True)

In [ ]:
train_df = train_df.dropna()
test_df = test_df.dropna()

len(train_df[train_df["store"] == store_id]) , len(test_df[test_df["store"] == store_id])

(111, 28)

In [ ]:
df.columns

Index(['store', 'weekly_sales', 'holiday_flag', 'temperature', 'fuel_Price',
       'cpi', 'unemployment'],
      dtype='object')

In [ ]:
train_df = train_df.sample(frac=1, random_state=42)

**1.1.  Random Forest**

In [ ]:
features = ["store" , "holiday_flag" , "weekly_sales(t-1)" , "temperature" , "fuel_Price" , "cpi" , "unemployment", "weekly_sales(t-2)" , "weekly_sales(t-3)" , "weekly_sales(t-4)"]

X_train , y_train =  train_df[features] , train_df["weekly_sales"]
X_test , y_test =  test_df[features] , test_df["weekly_sales"]

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_regressor.predict(X_test)

MeanAbsoluteError()(y_test , y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=49669.70376801589>

In [ ]:
for name, score in zip(features , rf_regressor.feature_importances_):
    print(name, score)

store 0.0019370535317163402
holiday_flag 0.007069219658232236
weekly_sales(t-1) 0.8214391078113474
temperature 0.00625446395457136
fuel_Price 0.002336057200027073
cpi 0.0033790838481882524
unemployment 0.0031708129160667783
weekly_sales(t-2) 0.05374352192542035
weekly_sales(t-3) 0.0109418384016592
weekly_sales(t-4) 0.08972884075277096


**1.2.  GradientBoostingRegressor**

In [ ]:
features = ["store" , "holiday_flag" , "weekly_sales(t-1)" , "weekly_sales(t-2)" , "weekly_sales(t-3)" , "weekly_sales(t-4)"]

X_train , y_train =  train_df[features] , train_df["weekly_sales"]
X_test , y_test =  test_df[features] , test_df["weekly_sales"]

In [ ]:
gb_regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_regressor.fit(X_train, y_train)

y_pred = gb_regressor.predict(X_test)

MeanAbsoluteError()(y_test , y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=49308.53936597178>

In [ ]:
for name, score in zip(features , gb_regressor.feature_importances_):
    print(name, score)

store 9.25296058815034e-05
holiday_flag 0.004626116994959973
weekly_sales(t-1) 0.6717706758876625
weekly_sales(t-2) 0.14466254185270935
weekly_sales(t-3) 0.004093221137763202
weekly_sales(t-4) 0.1747549145210235


**1.3.  Linear Regression**

In [ ]:
features = ["weekly_sales(t-1)" , "weekly_sales(t-4)"]

X_train , y_train =  train_df[features] , train_df["weekly_sales"]
X_test , y_test =  test_df[features] , test_df["weekly_sales"]

In [ ]:
L_regressor = LinearRegression()
L_regressor.fit(X_train, y_train)

y_pred = L_regressor.predict(X_test)

MeanAbsoluteError()(y_test , y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=44670.29831845118>

**1.4.  Voting Regressor**

In [ ]:
features = ["store" , "holiday_flag" , "weekly_sales(t-1)" , "weekly_sales(t-2)" , "weekly_sales(t-3)" , "weekly_sales(t-4)"]

X_train , y_train =  train_df[features] , train_df["weekly_sales"]
X_test , y_test =  test_df[features] , test_df["weekly_sales"]

In [ ]:
voting_regressor = VotingRegressor(estimators=[
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=100, random_state=42))
])

# Train the VotingRegressor
voting_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = voting_regressor.predict(X_test)

MeanAbsoluteError()(y_test , y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=47709.77641161322>

**2.  Deep Learning based approach**

**2.1.  Heplful functions: Data Scaling + Windowing time series data.**

In this project we will be using the tf.data.Datasets instead of tensors to represent the dataset of study.

In [ ]:
#Min-Max Scaling
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

#Inverse MIN-MAX scaling
def inverse_normalize_series(data, min, max):
    data = data * max
    data = data + min
    return data

In [ ]:
#windowing train data
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset


#windowing test data
def windowed_test_dataset(series, window_size , batch_size):

   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)

   return ds

Just like for statistical models and ML based models, we for each store data we will reserve the first 80% for training and the last 20% for testing.

In [ ]:
warnings.filterwarnings("ignore")

batch_size = 16
window_size = 4
rows_train = 115
rows_test = 28

min , max = df["weekly_sales"].min() , df["weekly_sales"].max()
min , max


def empty_generator():
    yield from []

element_spec = (
    tf.TensorSpec(shape=(None, None), dtype=tf.float64),
    tf.TensorSpec(shape=(None,), dtype=tf.float64)
)
train_dataset = tf.data.Dataset.from_generator(empty_generator, output_signature=element_spec)


element_spec = (tf.TensorSpec(shape=(None, None), dtype=tf.float64))
test_dataset = tf.data.Dataset.from_generator(empty_generator, output_signature=element_spec)



train_df = pd.DataFrame(columns=df.columns)
test_df_for_windowing = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

# Iterate over unique store values
for store_id in df['store'].unique():

    store_data = df[df['store'] == store_id]

    # Split data for train and test
    train_data = store_data.head(rows_train)
    test_data_for_windowing = store_data.tail(rows_test+window_size)
    test_data = store_data.tail(rows_test)


    # # Append to train_df and test_df
    train_df = train_df.append(train_data)
    test_df_for_windowing = test_df_for_windowing.append(test_data_for_windowing)
    test_df = test_df.append(test_data)


    # Scaling the target with MIN-MAX scaling
    train_data["scaled_weekly_sales"] = normalize_series(train_data["weekly_sales"] , min , max)
    test_data_for_windowing["scaled_weekly_sales"] = normalize_series(test_data_for_windowing["weekly_sales"] , min , max)



    tr_dataset = windowed_dataset(train_data["scaled_weekly_sales"], window_size, batch_size, 1000)
    train_dataset = train_dataset.concatenate(tr_dataset)

    te_dataset = windowed_test_dataset(test_data_for_windowing["scaled_weekly_sales"], window_size , rows_test)
    test_dataset = test_dataset.concatenate(te_dataset)

In [ ]:
# df[df['store'] == 1]["weekly_sales"][-28:]

In [ ]:
# for feature in test_dataset.take(1):
#   print(feature)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu',strides=1, padding="causal" , input_shape=[None,1]),
  tf.keras.layers.LSTM(4 , activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(loss=MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=["mae"])  #MeanSquaredError() or huber() based on the data you used
history = model.fit(train_dataset, epochs=50 ,  verbose=1)

Epoch 1/50
315/315 [==============================] - 8s 13ms/step - loss: 0.0438 - mae: 0.1557
Epoch 2/50
315/315 [==============================] - 3s 11ms/step - loss: 0.0251 - mae: 0.1254
Epoch 3/50
315/315 [==============================] - 3s 8ms/step - loss: 0.0231 - mae: 0.1242
Epoch 4/50
315/315 [==============================] - 4s 13ms/step - loss: 0.0229 - mae: 0.1243
Epoch 5/50
315/315 [==============================] - 6s 17ms/step - loss: 0.0205 - mae: 0.1123
Epoch 6/50
315/315 [==============================] - 6s 18ms/step - loss: 0.0099 - mae: 0.0580
Epoch 7/50
315/315 [==============================] - 2s 7ms/step - loss: 0.0051 - mae: 0.0404
Epoch 8/50
315/315 [==============================] - 2s 7ms/step - loss: 0.0035 - mae: 0.0335
Epoch 9/50
315/315 [==============================] - 2s 7ms/step - loss: 0.0026 - mae: 0.0282
Epoch 10/50
315/315 [==============================] - 2s 7ms/step - loss: 0.0023 - mae: 0.0260
Epoch 11/50
315/315 [=======================

In [ ]:
scaled_predictions = model.predict(test_dataset)
predictions = inverse_normalize_series(scaled_predictions, min, max)


MeanAbsoluteError()(test_df["weekly_sales"] , predictions)

45/45 [==============================] - 2s 17ms/step


<tf.Tensor: shape=(), dtype=float32, numpy=615709.3>

We observe that the Deep Learning-based approach, consisting of the LSTM+1D-CNN architecture, exhibits the lowest performance among all the models employed, including naive, statistical, and machine learning models. This outcome is noteworthy since the LSTM+1D-CNN architecture is renowned for its ability to capture contextual patterns within time series data. However, this underperformance can be attributed to the limited size of our dataset. These sophisticated architectures typically excel with substantial amounts of data. Furthermore, the unique patterns inherent to each store's weekly sales contribute to this phenomenon. For optimal performance from this architecture, a substantial volume of data per store is required

**3.  Linear Model for each store**

Among all the ML-based models we used, the linear model is the only one that outperformed the the naive forecastiong models. In this section, for each store we will train a linear model to see if we can outperform the SARIMA model.

To compare it with SARIMA we will import the dataframe od MSE and MAE we created in the notebook statistical_modeling.ipynb.

In [ ]:
mae_df = pd.read_csv("statistical_modeling_MAE.csv")
mse_df = pd.read_csv("statistical_modeling_MSE.csv")


columns = ["store_id" , "mse_naive" , "mse_naive_seasonal" , "mse_sarima"]
mse_df.columns , mae_df.columns = columns , columns

In [ ]:
warnings.filterwarnings("ignore")

def calculate_r_squared(mse, y_true):
    # Calculate the Total Sum of Squares (SS_total)
    mean_y_true = np.mean(y_true)
    ss_total = np.sum((y_true - mean_y_true)**2)

    # Calculate R-squared using the formula
    r_squared = 1 - (mse / ss_total)

    return r_squared



rows_train = 115
rows_test = 28

mae_linear ,  mse_linear , r2_linear = [] , [] , []

# Iterate over unique store values
for store_id in tqdm_notebook(df['store'].unique()):
    store_data = df[df['store'] == store_id]


    # Some feature engineering
    store_data["weekly_sales(t-1)"] , store_data["weekly_sales(t-2)"] = store_data["weekly_sales"].shift(1) , store_data["weekly_sales"].shift(2)
    store_data["weekly_sales(t-3)"] , store_data["weekly_sales(t-4)"] = store_data["weekly_sales"].shift(3) , store_data["weekly_sales"].shift(4)


    # Split data for train and test
    train_data = store_data.head(rows_train)
    test_data = store_data.tail(rows_test)

    #Drop NAN values
    train_data = train_data.dropna()
    test_data = test_data.dropna()

    #shufll train data
    train_data = train_data.sample(frac=1, random_state=42)


    features = ["weekly_sales(t-1)" , "weekly_sales(t-2)" , "weekly_sales(t-3)" , "weekly_sales(t-4)"]
    X_train , y_train =  train_data[features] , train_data["weekly_sales"]
    X_test , y_test =  test_data[features] , test_data["weekly_sales"]


    L_regressor = LinearRegression()
    L_regressor.fit(X_train, y_train)


    y_pred = L_regressor.predict(X_test)
    mae_linear.append(MeanAbsoluteError()(y_test , y_pred).numpy())
    mse_linear.append(MeanSquaredError()(y_test , y_pred).numpy())

    #if you are interested about the Coefficient of Determination R^2 (between 0 and 1)
    r2_linear.append(calculate_r_squared(MeanSquaredError()(y_test , y_pred).numpy() , y_test))

  0%|          | 0/45 [00:00<?, ?it/s]

In [ ]:
r2_linear[:5]

[0.9693563104354245,
 0.9629599258169561,
 0.9516209157886054,
 0.9625423293238476,
 0.9724657985599436]

In [ ]:
mae_df["mae_LR"] = mae_linear
mse_df["mse_LR"] = mse_linear

In [ ]:
mae_df.head()

,store_id,mse_naive,mse_naive_seasonal,mse_sarima,mae_LR
0,1,90136.727143,71970.351071,63623.810516,60529.426124
1,2,81483.281071,81267.170000,64559.441360,62166.180949
2,3,17841.880000,18993.431429,17014.648746,15147.082760
3,4,71920.641786,71340.272143,50918.344467,50668.029030
4,5,15007.310714,15163.618929,11661.809412,11063.370592


In [ ]:
mse_df.head()

,store_id,mse_naive,mse_naive_seasonal,mse_sarima,mse_LR
0,1,1.304006e+10,7.755212e+09,6.224669e+09,5.562569e+09
1,2,1.045874e+10,9.578124e+09,6.685684e+09,5.809893e+09
2,3,5.251918e+08,5.238562e+08,4.827320e+08,3.599329e+08
3,4,7.219025e+09,8.228034e+09,3.595561e+09,3.814089e+09
4,5,3.565480e+08,3.544499e+08,2.119441e+08,1.922405e+08


In [ ]:
mae_df.mean() , mae_df.median()

(store_id                 23.000000
 mse_naive             50734.819667
 mse_naive_seasonal    61100.378310
 mse_sarima            42750.153738
 mae_LR                45413.105853
 dtype: float64,
 store_id                 23.000000
 mse_naive             47525.538571
 mse_naive_seasonal    56400.374643
 mse_sarima            40900.278946
 mae_LR                40803.439314
 dtype: float64)

In [ ]:
mse_df.mean() , mse_df.median()

(store_id              2.300000e+01
 mse_naive             5.715255e+09
 mse_naive_seasonal    8.012645e+09
 mse_sarima            3.949630e+09
 mse_LR                4.783393e+09
 dtype: float64,
 store_id              2.300000e+01
 mse_naive             3.391680e+09
 mse_naive_seasonal    5.097732e+09
 mse_sarima            2.543941e+09
 mse_LR                2.644166e+09
 dtype: float64)

The SARIMA model has slightly outperformed the Linear model in both means of MAE and MSE.

In [ ]:
mse_df.to_csv("MSE_Models_Stores.csv")
mae_df.to_csv("MAE_Models_Stores.csv")

In [ ]:
overall_losses = pd.DataFrame()
overall_losses["mse_losses"] , overall_losses["mae_losses"] = mse_df.mean() , mse_df.mean()
overall_losses.drop(["store_id"] , inplace=True)

In [ ]:
overall_losses

,mse_losses,mae_losses
mse_naive,5.715255e+09,5.715255e+09
mse_naive_seasonal,8.012645e+09,8.012645e+09
mse_sarima,3.949630e+09,3.949630e+09
mse_LR,4.783393e+09,4.783393e+09


In [ ]:
overall_losses.to_csv("overall_losses.csv")